In [0]:
import pandas as pd
import itertools
import re
import nltk
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn import datasets, cluster
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# Luis' path
prods = pd.read_json("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/meta_Video_Games.json", lines = True)

In [0]:
# Sarah's path
# prods = pd.read_json("/content/drive/My Drive/ml_data/meta_Video_Games.json", lines = True)

In [0]:
prods.drop_duplicates(subset = "asin", keep = "first", inplace = True)

In [18]:
prods.shape

(71984, 17)

In [19]:
prods.head()

,category,title,image,brand,rank,main_cat,asin,description,also_buy,also_view,price,feature,date,tech1,details,similar_item,tech2
0,"[Video Games, PC, Games]",Reversi Sensory Challenger,[https://images-na.ssl-images-amazon.com/image...,Fidelity Electronics,">#2,623,937 in Toys & Games (See Top 100 in To...",Toys & Games,0042000742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Video Games, Xbox 360, Games, ]",Medal of Honor: Warfighter - Includes Battlefi...,[https://images-na.ssl-images-amazon.com/image...,by\n \n EA Games,">#67,231 in Video Games (See Top 100 in Video ...",Video Games,0078764343,[Brand new sealed!],[B00PADROYW],"[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",street fighter 2 II turbo super nintendo snes ...,NaN,Nintendo,">#134,433 in Video Games (See Top 100 in Video...",Video Games,0276425316,NaN,NaN,NaN,$0.72,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",Xbox 360 MAS STICK,[https://images-na.ssl-images-amazon.com/image...,by\n \n MAS SYSTEMS,">#105,263 in Video Games (See Top 100 in Video...",Video Games,0324411812,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,NaN,NaN,NaN,[Original PCB used from Xbox 360 Control Pad (...,NaN,NaN,NaN,NaN,NaN
4,"[Video Games, PC, Games, , Grades 2-12, Spelli...",Phonics Alive! 3: The Speller,[https://images-na.ssl-images-amazon.com/image...,by\n \n Advanced Software Pty. Ltd.,">#92,397 in Video Games (See Top 100 in Video ...",Video Games,0439335310,"[Phonics Alive! 3, The Speller teaches student...",NaN,[B000BCZ7U0],NaN,"[Grades 2-12, Spelling Program, Teaches Spelli...",NaN,NaN,NaN,NaN,NaN


In [20]:
sum(prods.description.isna())

8971

In [0]:
features = prods.dropna(subset = ['description', 'category', 'feature'])

In [23]:
[i.remove("Video Games") for i in features['category']]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [24]:
features['category']

3        [Xbox 360, Accessories, Controllers, Joysticks...
4        [PC, Games, , Grades 2-12, Spelling Program, T...
5                      [PC, Games, , Sim City 3000 CD-ROM]
8                                              [PC, Games]
9        [PC, Games, , Like the real world, Freddi Fish...
                               ...                        
84884    [Xbox One, Games, , Remastered in 1080p., BioS...
84887              [Kids & Family, PlayStation 4, Games, ]
84889    [PlayStation 4, Digital Games & DLC, Downloada...
84891                            [Xbox 360, Accessories, ]
84892                       [PC, Accessories, Gaming Mice]
Name: category, Length: 50544, dtype: object

In [25]:
features["game_cat"] = ["Games" in i for i in features['category']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
features = features[features.game_cat != False] 

In [27]:
features.head()

,category,title,image,brand,rank,main_cat,asin,description,also_buy,also_view,price,feature,date,tech1,details,similar_item,tech2,game_cat
4,"[PC, Games, , Grades 2-12, Spelling Program, T...",Phonics Alive! 3: The Speller,[https://images-na.ssl-images-amazon.com/image...,by\n \n Advanced Software Pty. Ltd.,">#92,397 in Video Games (See Top 100 in Video ...",Video Games,0439335310,"[Phonics Alive! 3, The Speller teaches student...",NaN,[B000BCZ7U0],NaN,"[Grades 2-12, Spelling Program, Teaches Spelli...",NaN,NaN,NaN,NaN,NaN,True
5,"[PC, Games, , Sim City 3000 CD-ROM]",Sim City 3000,[https://images-na.ssl-images-amazon.com/image...,by\n \n Scholastic,">#19,087 in Video Games (See Top 100 in Video ...",Video Games,0439339006,[CD-ROM],NaN,"[B00004T1M7, B00000G16I, B00006GF1U, B0000C0YW...",NaN,[Sim City 3000 CD-ROM],NaN,NaN,NaN,NaN,NaN,True
8,"[PC, Games]",Need for Speed Porsche Unleashed,[https://images-na.ssl-images-amazon.com/image...,Electronic Arts,">#51,505 in Video Games (See Top 100 in Video ...",Video Games,043933702X,[video game],NaN,"[B00004DITL, B000067NP1, B00003W869]",$49.98,[video game],NaN,NaN,NaN,NaN,NaN,True
9,"[PC, Games, , Like the real world, Freddi Fish...",Freddi Fish and the Case of The Missing Kelp S...,[https://images-na.ssl-images-amazon.com/image...,by\n \n Humongous Entertainment,">#58,523 in Video Games (See Top 100 in Video ...",Video Games,0439339987,[Grandma Groupers kelp seeds are missing and w...,"[B00002S6E5, B000NQI666, B000AQJ4AG, B000FETCF...","[B000FETCF0, B00005BU3M, B00004UCUT, B0009RAVF...",NaN,"[Like the real world, Freddi Fishs world chang...",NaN,NaN,NaN,NaN,NaN,True
10,"[PC, Games]",Ice Cream Truck,[https://images-na.ssl-images-amazon.com/image...,Sunburtst,">#114,262 in Video Games (See Top 100 in Video...",Video Games,0439342260,[This software is BRAND NEW. Packaging may dif...,NaN,NaN,NaN,"[8 MB RAM recommended, 486/DX2-66 MHz processo...",NaN,NaN,NaN,NaN,NaN,True


In [0]:
features.dropna(subset=["description"],inplace = True)

In [0]:
# from the feature column, join the features into one big string
feat_list = []

for r in features.description:
  j = ' '.join(r)
  feat_list.append(j)

In [31]:
feat_list[0:10]

['Phonics Alive! 3, The Speller teaches students to plan how to spell new and unfamiliar words. Traditional Spelling Rules are explained and actively taught, followed by practice activities. Sight Words are taught and practice games ensure that learning is fun. To best suit individual needs, students are able to choose from a variety of sight words (general lists, community signs, science words, mathematics words, secondary words or compound words). A wide variety of additional words are provided with sound files enabling teachers to add custom word lists to the program. Students will advance their understanding of the grapho-phonic nature of our language and of the complicated alphabetic mapping system of English. The program consists of 15 modules. It is self paced and sequentional. The starting age of the program is 8 or grade 3. Phonics Alive! 3, The Speller continues the tradition of excellence in offering fun learning through a high quality teacher resource. The student is again 

In [32]:
len(np.array(feat_list))

27256

In [0]:
fdf = pd.DataFrame(np.array(feat_list))

In [34]:
len(features.description)

27256

In [0]:
fdf.columns = ['feat_list']

In [36]:
fdf.head()

,feat_list
0,"Phonics Alive! 3, The Speller teaches students..."
1,CD-ROM
2,video game
3,Grandma Groupers kelp seeds are missing and wi...
4,This software is BRAND NEW. Packaging may diff...


In [0]:
features['feat_list'] = fdf['feat_list']

In [38]:
features.head()

,category,title,image,brand,rank,main_cat,asin,description,also_buy,also_view,price,feature,date,tech1,details,similar_item,tech2,game_cat,feat_list
4,"[PC, Games, , Grades 2-12, Spelling Program, T...",Phonics Alive! 3: The Speller,[https://images-na.ssl-images-amazon.com/image...,by\n \n Advanced Software Pty. Ltd.,">#92,397 in Video Games (See Top 100 in Video ...",Video Games,0439335310,"[Phonics Alive! 3, The Speller teaches student...",NaN,[B000BCZ7U0],NaN,"[Grades 2-12, Spelling Program, Teaches Spelli...",NaN,NaN,NaN,NaN,NaN,True,This software is BRAND NEW. Packaging may diff...
5,"[PC, Games, , Sim City 3000 CD-ROM]",Sim City 3000,[https://images-na.ssl-images-amazon.com/image...,by\n \n Scholastic,">#19,087 in Video Games (See Top 100 in Video ...",Video Games,0439339006,[CD-ROM],NaN,"[B00004T1M7, B00000G16I, B00006GF1U, B0000C0YW...",NaN,[Sim City 3000 CD-ROM],NaN,NaN,NaN,NaN,NaN,True,OREGON TRAIL 5TH EDITION (JEWEL CASE). Create ...
8,"[PC, Games]",Need for Speed Porsche Unleashed,[https://images-na.ssl-images-amazon.com/image...,Electronic Arts,">#51,505 in Video Games (See Top 100 in Video ...",Video Games,043933702X,[video game],NaN,"[B00004DITL, B000067NP1, B00003W869]",$49.98,[video game],NaN,NaN,NaN,NaN,NaN,True,There is so much to do on LEGO Island: buildin...
9,"[PC, Games, , Like the real world, Freddi Fish...",Freddi Fish and the Case of The Missing Kelp S...,[https://images-na.ssl-images-amazon.com/image...,by\n \n Humongous Entertainment,">#58,523 in Video Games (See Top 100 in Video ...",Video Games,0439339987,[Grandma Groupers kelp seeds are missing and w...,"[B00002S6E5, B000NQI666, B000AQJ4AG, B000FETCF...","[B000FETCF0, B00005BU3M, B00004UCUT, B0009RAVF...",NaN,"[Like the real world, Freddi Fishs world chang...",NaN,NaN,NaN,NaN,NaN,True,3 Great games in one box set!
10,"[PC, Games]",Ice Cream Truck,[https://images-na.ssl-images-amazon.com/image...,Sunburtst,">#114,262 in Video Games (See Top 100 in Video...",Video Games,0439342260,[This software is BRAND NEW. Packaging may dif...,NaN,NaN,NaN,"[8 MB RAM recommended, 486/DX2-66 MHz processo...",NaN,NaN,NaN,NaN,NaN,True,cartoon network


In [0]:
features.reset_index(inplace=True)

In [40]:
features.head()

,index,category,title,image,brand,rank,main_cat,asin,description,also_buy,also_view,price,feature,date,tech1,details,similar_item,tech2,game_cat,feat_list
0,4,"[PC, Games, , Grades 2-12, Spelling Program, T...",Phonics Alive! 3: The Speller,[https://images-na.ssl-images-amazon.com/image...,by\n \n Advanced Software Pty. Ltd.,">#92,397 in Video Games (See Top 100 in Video ...",Video Games,0439335310,"[Phonics Alive! 3, The Speller teaches student...",NaN,[B000BCZ7U0],NaN,"[Grades 2-12, Spelling Program, Teaches Spelli...",NaN,NaN,NaN,NaN,NaN,True,This software is BRAND NEW. Packaging may diff...
1,5,"[PC, Games, , Sim City 3000 CD-ROM]",Sim City 3000,[https://images-na.ssl-images-amazon.com/image...,by\n \n Scholastic,">#19,087 in Video Games (See Top 100 in Video ...",Video Games,0439339006,[CD-ROM],NaN,"[B00004T1M7, B00000G16I, B00006GF1U, B0000C0YW...",NaN,[Sim City 3000 CD-ROM],NaN,NaN,NaN,NaN,NaN,True,OREGON TRAIL 5TH EDITION (JEWEL CASE). Create ...
2,8,"[PC, Games]",Need for Speed Porsche Unleashed,[https://images-na.ssl-images-amazon.com/image...,Electronic Arts,">#51,505 in Video Games (See Top 100 in Video ...",Video Games,043933702X,[video game],NaN,"[B00004DITL, B000067NP1, B00003W869]",$49.98,[video game],NaN,NaN,NaN,NaN,NaN,True,There is so much to do on LEGO Island: buildin...
3,9,"[PC, Games, , Like the real world, Freddi Fish...",Freddi Fish and the Case of The Missing Kelp S...,[https://images-na.ssl-images-amazon.com/image...,by\n \n Humongous Entertainment,">#58,523 in Video Games (See Top 100 in Video ...",Video Games,0439339987,[Grandma Groupers kelp seeds are missing and w...,"[B00002S6E5, B000NQI666, B000AQJ4AG, B000FETCF...","[B000FETCF0, B00005BU3M, B00004UCUT, B0009RAVF...",NaN,"[Like the real world, Freddi Fishs world chang...",NaN,NaN,NaN,NaN,NaN,True,3 Great games in one box set!
4,10,"[PC, Games]",Ice Cream Truck,[https://images-na.ssl-images-amazon.com/image...,Sunburtst,">#114,262 in Video Games (See Top 100 in Video...",Video Games,0439342260,[This software is BRAND NEW. Packaging may dif...,NaN,NaN,NaN,"[8 MB RAM recommended, 486/DX2-66 MHz processo...",NaN,NaN,NaN,NaN,NaN,True,cartoon network


In [41]:
features["description_joined"] = features.description.str.join(" ")
features["description_joined"]

0        Phonics Alive! 3, The Speller teaches students...
1                                                   CD-ROM
2                                               video game
3        Grandma Groupers kelp seeds are missing and wi...
4        This software is BRAND NEW. Packaging may diff...
                               ...                        
27251    OlliOlli: Epic Combo Edition includes a gold l...
27252     Return to the cities of Rapture and Columbia ...
27253    Experience explosive airborne action, high in ...
27254     Return to the cities of Rapture and Columbia ...
27255    All-new game modes, new and deeper experiences...
Name: description_joined, Length: 27256, dtype: object

In [0]:
features.to_csv("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/descriptions_clean.csv")

In [0]:
feat_idx = features.columns.get_loc('feat_list')

In [0]:
# pull out 10 of 50 most common words for categorizing

In [0]:
features.dropna(subset = ['feat_list'], inplace = True)

In [0]:
len(features.feat_list)

38479

In [0]:
for i, row in features.iterrows():
  i_val = [nltk.tokenize.word_tokenize(i) for i in str(features.iloc[i, feat_idx]).split(' ')]
  features.at[i,"feat_list"] = [j for i in i_val for j in i]

In [0]:
features_clean = features['feat_list']
features_clean = features_clean.apply(lambda x: [i for i in x if len(i) > 1])
features_clean = features_clean.apply(lambda x: [i for i in x if i.isalpha()])
stopwords = set(nltk.corpus.stopwords.words('english'))
features_clean = features_clean.apply(lambda x: [i for i in x if i not in stopwords])

In [0]:
len(features)

12610

In [0]:
f_clean = np.array(features_clean)
f_clean = list(itertools.chain.from_iterable(f_clean))
fdist = nltk.FreqDist(f_clean)
fdist.most_common(500)

[('game', 4374),
 ('new', 3997),
 ('The', 2547),
 ('players', 2137),
 ('play', 1790),
 ('characters', 1753),
 ('world', 1623),
 ('Play', 1508),
 ('action', 1488),
 ('different', 1453),
 ('including', 1435),
 ('levels', 1394),
 ('unique', 1386),
 ('weapons', 1378),
 ('New', 1331),
 ('games', 1228),
 ('player', 1225),
 ('gameplay', 1194),
 ('Game', 1165),
 ('like', 1158),
 ('mode', 1127),
 ('modes', 1103),
 ('enemies', 1082),
 ('multiplayer', 1036),
 ('missions', 1021),
 ('system', 976),
 ('time', 922),
 ('combat', 906),
 ('battle', 901),
 ('Use', 897),
 ('online', 891),
 ('control', 868),
 ('one', 868),
 ('skills', 860),
 ('use', 850),
 ('experience', 815),
 ('graphics', 771),
 ('For', 770),
 ('friends', 766),
 ('character', 755),
 ('environments', 751),
 ('team', 743),
 ('Mode', 732),
 ('take', 702),
 ('way', 698),
 ('moves', 682),
 ('even', 651),
 ('Choose', 620),
 ('Over', 617),
 ('get', 611),
 ('special', 608),
 ('adventure', 602),
 ('original', 602),
 ('abilities', 589),
 ('You', 5

In [0]:
common = fdist.most_common(500)

In [0]:
attr500 = {}

for k in common:
  attr500.update({k[0].lower(): []})


In [0]:
len(attr500.keys())

430

In [0]:
# if the list in the cell contains this word
# lowercase the list of the words

In [0]:
for r in features.feat_list:
  j = [i.lower() for i in r]
  l = [i.lower() + 's' for i in r]
  for k, v in attr500.items():
    if (k in j) | (k in l):
      attr500[k].append(1)
    else:
      attr500[k].append(0)

In [0]:
attr_df = pd.DataFrame(attr500)

In [0]:
# run PCA on attr_df
pca = PCA(50)

In [0]:
pca.fit(attr_df)

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [0]:
print(sum(pca.explained_variance_ratio_))

0.5133010177431296


In [0]:
agglo = cluster.FeatureAgglomeration(n_clusters=8)

In [0]:
agglo.fit(attr_df)

FeatureAgglomeration(affinity='euclidean', compute_full_tree='auto',
                     connectivity=None, distance_threshold=None, linkage='ward',
                     memory=None, n_clusters=8,
                     pooling_func=<function mean at 0x7fb0f4629950>)

In [0]:
attr_reduced = agglo.transform(attr_df)

In [0]:
pd.DataFrame(attr_reduced)

,0,1,2,3,4,5,6,7
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.005698,0.0
2,0.051724,0.0,0.333333,1.0,0.0,0.0,0.008547,0.0
3,0.120690,0.4,0.000000,0.8,1.0,1.0,0.028490,0.0
4,0.000000,0.0,0.000000,0.2,0.0,0.0,0.014245,0.0
...,...,...,...,...,...,...,...,...
12605,0.068966,0.6,0.166667,0.6,0.0,0.0,0.011396,0.0
12606,0.155172,0.4,0.833333,1.0,1.0,0.0,0.022792,0.0
12607,0.241379,0.4,0.500000,0.8,0.0,1.0,0.048433,0.0
12608,0.017241,0.2,0.333333,0.2,1.0,0.0,0.005698,0.0
